In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import pathlib

from credentials import SAMPLE_DB, FULL_DB
from librarian import SAMPLE_DATA_DIR

import encoders
import db_connect
import helpers

2024-11-19 21:21:24.689912: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-19 21:21:24.700347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-19 21:21:24.711536: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-19 21:21:24.715409: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-19 21:21:24.724885: I tensorflow/core/platform/cpu_feature_guar

In [2]:
DATA_DIR = SAMPLE_DATA_DIR
CHUNK_SIZE = 128
MAIN_ACCURACY_METRIC = "mae"
GOOD_ACCURACY_DIRECTION = "min" # Some accuracy metrics go up for better results, some go down

ENCRYPTED_FILE_LIMIT = 0 # -1 to disable limit
SPLIT_SEED = 42
SPLIT_TEST_SIZE = 0.25     # 0.25 is default

LOAD_BEST_MODEL = False # If False, a new model will be created from scratch
SAVE_BEST_MODEL = True
TRAIN_MODEL = True
EPOCHS = 5

db = db_connect.DB(SAMPLE_DB)

In [3]:
# Get database IDs for encoders and key types

encoder_ids= {}
key_type_ids = {}

with db.get_session() as session:
    for encoder in encoders.ALL_ENCODER_NAMES:
        id = db.get_encoder_id(session, encoder)
        encoder_ids[encoder] = id

    print(f"Encoder IDs: {encoder_ids}")

    for key_type in encoders.KEY_NAMES:
        id = db.get_key_type_id(session, key_type)
        key_type_ids[key_type] = id

    print(f"Key Type IDs: {key_type_ids}")

Encoder IDs: {'None': 1, 'Simplifier': 2, 'Caesar Cipher': 3, 'Substitution Cipher': 4, 'Enigma Machine': 5}
Key Type IDs: {'Character Offset': 1, 'Character Map': 2, 'Rotor Settings': 3}


In [4]:
# Map source ID to plaintext file (1) details, and source ID to corresponding ciphertext files (1+) details
sid_to_p = {}
sid_to_c = {}

cipher_id = encoder_ids[encoders.ENCODER_CAESAR]
with db.get_session() as session:
    # Get all files encrypted with the cipher we care about
    encrypted_files = db.get_files_by_source_and_encoder(session, -1, cipher_id)

    if len(encrypted_files) > ENCRYPTED_FILE_LIMIT and ENCRYPTED_FILE_LIMIT > 0:
        encrypted_files = encrypted_files[0:ENCRYPTED_FILE_LIMIT]

    for c in encrypted_files:
        sid = c.source_id
    
        if sid not in sid_to_p:
            plaintext_ids = db.get_files_by_source_and_encoder(session, sid, encoder_ids[encoders.ENCODER_SIMPLIFIER])
            if len(plaintext_ids) != 1:
                raise Exception(f"Found {len(plaintext_ids)} plaintexts for source ID {sid}; should be exactly 1")
            sid_to_p[sid] = plaintext_ids[0]

        if sid not in sid_to_c:
            sid_to_c[sid] = []
        sid_to_c[sid].append(c)

len(sid_to_p), len(sid_to_c)

(6, 6)

In [5]:
# Build up the features (X, the cipher texts as values) and targets (y, the plain texts as values).
# Note the targets can be repeated since a plaintext can be encrypted repeatedly by different keys.
X = []
y = []

for sid in sid_to_p:
    plaintext = helpers.read_text_file(sid_to_p[sid].path)
    target_chunks = helpers.string_to_bytes(plaintext, CHUNK_SIZE)

    for c in sid_to_c[sid]:
        ciphertext = helpers.read_text_file(c.path)
        feature_chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)

        if len(target_chunks) != len(feature_chunks):
            raise Exception(f"Chunk count mismatch; {len(target_chunks)} != {len(feature_chunks)}")

        for i in range (len(target_chunks)):
            X.append(feature_chunks[i])
            y.append(target_chunks[i])

X = np.array(X)
y = np.array(y)

X.shape, y.shape, X[0].shape, y[0].shape

((772960, 128), (772960, 128), (128,), (128,))

In [6]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SPLIT_TEST_SIZE, random_state=SPLIT_SEED)
print( len(X), len(y), len(X_train), len(X_test), len(y_train), len(y_test) )

772960 772960 579720 193240 579720 193240


In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((579720, 128), (193240, 128))

In [8]:
if LOAD_BEST_MODEL:
    print(f"Loading model from {BEST_PATH}")
    nn = tf.keras.models.load_model(BEST_PATH)
else:
    print("Building new model")
    in_shape = (CHUNK_SIZE,)
    nn = tf.keras.models.Sequential()
    
    # Input layer
    nn.add(tf.keras.Input(shape=in_shape))
    
    # Hidden layers
    activations = ["tanh", "relu", "elu", "exponential", "gelu", "mish", "relu6", "tanh", "selu"]
    unit_counts = [CHUNK_SIZE*2, CHUNK_SIZE*2]
    for u in unit_counts:
        for a in activations:
            nn.add(tf.keras.layers.Dense(units=u, activation=a))
    
    # Output layer
    nn.add(tf.keras.layers.Dense(units=CHUNK_SIZE))
    
# Check the structure of the model
nn.summary()

Building new model


I0000 00:00:1732080111.336670    8632 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1732080111.490219    8632 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1732080111.490342    8632 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1732080111.494224    8632 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1732080111.494335    8632 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │        32,896 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,184,384 (4.52 MB)

 Trainable params: 1,184,384 (4.52 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Set up training checkpoint to save after each epoch, if it is a new best model:
BEST_PATH = './best.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=BEST_PATH,
    monitor=MAIN_ACCURACY_METRIC,
    mode=GOOD_ACCURACY_DIRECTION,
    save_best_only=True,
    save_weights_only=False,
    verbose=1)

if TRAIN_MODEL:
    print(f"Training model")
    
    # Compile the Sequential model together and customize metrics
    nn.compile(loss="mean_squared_error", optimizer="adamax", metrics=[MAIN_ACCURACY_METRIC])
    
    # Fit the model to the training data
    callbacks = None
    if SAVE_BEST_MODEL:
        callbacks = [model_checkpoint_callback]
    fit_model = nn.fit(X_train_scaled, y_train, epochs=EPOCHS, callbacks=callbacks)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Training model
Epoch 1/5


I0000 00:00:1732080133.252057    8787 service.cc:146] XLA service 0x7f6748023610 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732080133.253643    8787 service.cc:154]   StreamExecutor device (0): NVIDIA RTX 2000 Ada Generation Laptop GPU, Compute Capability 8.9
2024-11-19 21:22:13.431407: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-19 21:22:13.887203: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101
2024-11-19 21:22:20.831780: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1501', 24 bytes spill stores, 24 bytes spill loads

2024-11-19 21:22:21.232766: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to loc

   42/18117 ━━━━━━━━━━━━━━━━━━━━ 1:09 4ms/step - loss: 3984.0952 - mae: 59.5620   

I0000 00:00:1732080149.668135    8787 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


18112/18117 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 426.5924 - mae: 15.7495

2024-11-19 21:23:48.403192: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_523', 16 bytes spill stores, 24 bytes spill loads



18117/18117 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 426.5814 - mae: 15.7493
Epoch 1: mae improved from inf to 15.22063, saving model to ./best.keras
18117/18117 ━━━━━━━━━━━━━━━━━━━━ 111s 5ms/step - loss: 426.5792 - mae: 15.7493
Epoch 2/5
18117/18117 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 380.3316 - mae: 15.1335
Epoch 2: mae improved from 15.22063 to 15.13035, saving model to ./best.keras
18117/18117 ━━━━━━━━━━━━━━━━━━━━ 86s 5ms/step - loss: 380.3316 - mae: 15.1335
Epoch 3/5
18113/18117 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 380.1422 - mae: 15.1285
Epoch 3: mae improved from 15.13035 to 15.12649, saving model to ./best.keras
18117/18117 ━━━━━━━━━━━━━━━━━━━━ 83s 5ms/step - loss: 380.1422 - mae: 15.1285
Epoch 4/5
18106/18117 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 380.0174 - mae: 15.1244
Epoch 4: mae improved from 15.12649 to 15.12408, saving model to ./best.keras
18117/18117 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step - loss: 380.0174 - mae: 15.1244
Epoch 5/5
18113/18117 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
def decode_chunks_with_model(chunks: list[list], model, scaler, input_already_scaled = True) -> list[list]:
    if input_already_scaled:
        return model.predict(chunks)
    else:
        return model.predict(scaler.transform(chunks))

def decode_text_with_model(ciphertext: str, model, scaler) -> str:
    chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)
    decoded_chunks = decode_chunks_with_model(chunks, model, scaler, input_already_scaled = False)
    return helpers.bytes_to_string(decoded_chunks)

ciphertext_path = sid_to_c[list(sid_to_c.keys())[0]][0].path
ciphertext = helpers.read_text_file(ciphertext_path)
ciphertext = ciphertext[0:CHUNK_SIZE*2]
decode_text_with_model(ciphertext, nn, X_scaler)